In [ ]:
import sqlite3
from sqlite3 import Error

In [ ]:
con = sqlite3.connect('mydatabase.db')

In [ ]:
cursorObj = con.cursor()

In [ ]:
def sql_connection():

    try:

        con = sqlite3.connect(':memory:')

        print("Connection is established: Database is created in memory")

    except Error:

        print(Error)

    finally:

        con.close()

sql_connection()

In [ ]:
def sql_connection():

    try:

        con = sqlite3.connect('mydatabase.db')

        return con

    except Error:

        print(Error)

def sql_table(con):

    cursorObj = con.cursor()

    cursorObj.execute("CREATE TABLE employees(id integer PRIMARY KEY, name text, salary real, department text, position text, hireDate text)")

    con.commit()

con = sql_connection()

sql_table(con)

In [ ]:
cursorObj.execute("INSERT INTO employees VALUES(1, 'John', 700, 'HR', 'Manager', '2017-01-04')")

con.commit()

In [ ]:
!ls bank-additional/bank-additional

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def create_db(db_name):
    from pathlib import Path
    Path(db_name).touch()
    
def establish_db_connection(db_name):
    connection = sqlite3.connect(db_name)
    return connection

def create_and_populate_table(csv_file, connection, db_name, colnames, table="data"):
    c = connection.cursor()
#   c.execute(f'''DROP TABLE IF EXISTS {table}''')
    c.execute(f'''CREATE TABLE {table} {colnames}''')
    csv_file.to_sql(table, connection, if_exists='append', index = False)
    c.close()

In [ ]:
df = pd.read_csv("bank-additional/bank-additional-full.csv", sep=";")

In [ ]:
db_name = 'data.db'
colnames = tuple(df.columns)

create_db(db_name)
connection = establish_db_connection(db_name)
create_and_populate_table(df, connection, db_name, colnames)

In [ ]:
pd.read_sql('''SELECT age, duration, campaign, pdays
                      FROM data WHERE age > 20''',connection)

In [ ]:
np.random.randint(41188)

### Do some LGBM here

In [ ]:
import lightgbm as lgb

In [ ]:
data = pd.read_sql('''SELECT * FROM data''',connection)

In [ ]:
obj_feat = list(data.loc[:, data.dtypes == 'object'].columns.values)
obj_feat

In [ ]:
data.head()

In [ ]:
for feature in obj_feat:
    data[feature] = pd.Series(data[feature], dtype="category")

In [ ]:
def train_validate_test_split(df, train_percent=.8, validate_percent=.2, seed=None, 
                              target="cons.conf.idx", categorical=None):
    
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:]]
    
    y_train = train[target]
    y_test = validate[target]
    
    train.drop([target]+categorical, axis=1, inplace=True)
    validate.drop([target]+categorical, axis=1, inplace=True)
    
    return train, y_train, validate, y_test

In [ ]:
X_train, y_train, X_test, y_test = train_validate_test_split(data, categorical=obj_feat)

In [ ]:
X_train.columns

In [ ]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l2', 'mse'],
    'learning_rate': 0.005,
    "num_leaves": 20,  
    "max_bin": 128,
}
lgb_train = lgb.Dataset(X_train, y_train)
gbm = lgb.train(hyper_params, lgb_train, num_boost_round=10)

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

dtrain = xgb.DMatrix(data=X_train, label=y_train, enable_categorical=True)
dval = xgb.DMatrix(data=X_test, label=y_test, enable_categorical=True)
# dtest = xgb.DMatrix(data=xtest)

params = dict(objective ='reg:linear', colsample_bytree = 0.2, learning_rate = 0.1,
            max_depth = 3, alpha = 10, n_estimators = 10, eval_metric='rmse', verbosity=2)

num_round = 50
bst = xgb.train(params, dtrain, num_round, evals=[(dtrain, 'dtrain'), (dval, 'dval')])


In [ ]:
# training
model = xgb.XGBRegressor(colsample_bytree = 0.2, learning_rate = 0.1,
            max_depth = 3, alpha = 10, n_estimators = 10, eval_metric='rmse', verbosity=2)

model.fit(X_train, y_train, 
          eval_set=[(X_train, y_train), (X_test, y_test)], 
          early_stopping_rounds=200)

In [ ]:
# training
# save to JSON
model.save_model("model.json")

In [ ]:
# load the model
model2 = xgb.XGBRegressor()
model2.load_model("model.json")

In [ ]:
model2.predict(X_test)

### creating a databse with many tables

In [1]:
!rm -rf "db/multi_table.db"

import sqlite3
from sqlite3 import Error

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def create_db(db_name):
    from pathlib import Path
    Path(db_name).touch()
    
def establish_db_connection(db_name):
    connection = sqlite3.connect(db_name)
    return connection

def create_and_populate_table(csv_file, connection, db_name, colnames, table="data"):
    c = connection.cursor()
#     c.execute(f'''DROP TABLE IF EXISTS {table}''')
    try:
        c.execute(f'''CREATE TABLE {table} {colnames}''')
    except Exception as e:
        print(e)
    csv_file.to_sql(table, connection, if_exists='append', index = False)
    c.close()

In [4]:
db_name = "db/multi_table.db"
conn = sqlite3.connect(db_name)
c = conn.cursor()

In [5]:
!ls Ipin2016Dataset/

measure1_smartphone_sens.csv  measure2_phone_sens.csv	    PointsMapping.ods
measure1_smartphone_wifi.csv  measure2_smartphone_wifi.csv  README
measure1_smartwatch_sens.csv  measure2_timestamp_id.csv
measure1_timestamp_id.csv     measure2_watch_sens.csv


In [6]:
pointspd = pd.read_excel("Ipin2016Dataset/PointsMapping.ods", engine="odf")
columns = tuple(pointspd.columns)
columns

('ID', 'X', 'Y')

In [7]:
create_and_populate_table(pointspd, conn, db_name, columns, "pointsmapping")

In [8]:
colnames = ['arrival', 'departure', 'tid']
timestamp_id_1 = pd.read_csv("Ipin2016Dataset/measure1_timestamp_id.csv", names=colnames, header=None)
columns = tuple(timestamp_id_1.columns)

In [9]:
create_and_populate_table(timestamp_id_1, conn, db_name, columns, "timestamp")

In [10]:
colnames = ['arrival', 'departure', 'tid']
timestamp_id_1 = pd.read_csv("Ipin2016Dataset/measure2_timestamp_id.csv", names=colnames, header=None)
columns = tuple(timestamp_id_1.columns)

In [11]:
create_and_populate_table(timestamp_id_1, conn, db_name, columns, "timestamp")

table timestamp already exists


In [12]:
smartphone_sens1 = pd.read_csv("Ipin2016Dataset/measure1_smartphone_sens.csv")
oldcolumns =  tuple(smartphone_sens1.columns)
columns = tuple([col.strip() for col in oldcolumns])
# smartphone_sens1 = pd.read_csv("Ipin2016Dataset/measure1_smartphone_sens.csv")
smartphone_sens1.rename(columns = dict([(key, value) for key, value in zip(oldcolumns, columns)]), inplace=True)

In [13]:
create_and_populate_table(smartphone_sens1, conn, db_name, columns, "smartphone_sensor")

In [14]:
smartphone_sens1 = pd.read_csv("Ipin2016Dataset/measure2_phone_sens.csv", names=columns)
create_and_populate_table(smartphone_sens1, conn, db_name, columns, "smartphone_sensor2")

In [15]:
smartphone_wifi1 = pd.read_csv("Ipin2016Dataset/measure1_smartphone_wifi.csv", names=columns)
create_and_populate_table(smartphone_wifi1, conn, db_name, columns, "smartphone_wifi")

In [16]:
smartphone_wifi1 = pd.read_csv("Ipin2016Dataset/measure2_smartphone_wifi.csv", names=columns)
create_and_populate_table(smartphone_wifi1, conn, db_name, columns, "smartphone_wifi")

table smartphone_wifi already exists


In [17]:
watch_sens = pd.read_csv("Ipin2016Dataset/measure1_smartwatch_sens.csv", names=columns)
create_and_populate_table(watch_sens, conn, db_name, columns, "smartwatch_sensor")

In [18]:
watch_sens = pd.read_csv("Ipin2016Dataset/measure2_watch_sens.csv", names=columns)
create_and_populate_table(watch_sens, conn, db_name, columns, "smartwatch_sensor")

table smartwatch_sensor already exists


In [19]:
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""

c.execute(sql_query)

print(c.fetchall())

[('pointsmapping',), ('timestamp',), ('smartphone_sensor',), ('smartphone_sensor2',), ('smartphone_wifi',), ('smartwatch_sensor',)]


In [20]:
sql_query = '''SELECT * FROM pointsmapping
INNER JOIN timestamp ON timestamp.tid = pointsmapping.ID;'''

label_df = pd.read_sql_query(sql_query, conn)

In [21]:
label_df.describe()

,ID,X,Y,arrival,departure,tid
count,646.000000,646.000000,646.000000,6.460000e+02,6.460000e+02,646.000000
mean,161.043344,24.601858,13.770279,1.422880e+12,1.422880e+12,161.043344
std,92.871561,11.602004,10.817825,1.713053e+08,1.713050e+08,92.871561
min,2.000000,0.600000,0.600000,1.422629e+12,1.422629e+12,2.000000
25%,79.250000,17.400000,5.400000,1.422632e+12,1.422632e+12,79.250000
50%,160.000000,30.000000,6.600000,1.422885e+12,1.422885e+12,160.000000
75%,240.750000,35.400000,25.200000,1.423044e+12,1.423044e+12,240.750000
max,324.000000,36.000000,34.200000,1.423128e+12,1.423128e+12,324.000000


In [22]:
label_df.describe()

,ID,X,Y,arrival,departure,tid
count,646.000000,646.000000,646.000000,6.460000e+02,6.460000e+02,646.000000
mean,161.043344,24.601858,13.770279,1.422880e+12,1.422880e+12,161.043344
std,92.871561,11.602004,10.817825,1.713053e+08,1.713050e+08,92.871561
min,2.000000,0.600000,0.600000,1.422629e+12,1.422629e+12,2.000000
25%,79.250000,17.400000,5.400000,1.422632e+12,1.422632e+12,79.250000
50%,160.000000,30.000000,6.600000,1.422885e+12,1.422885e+12,160.000000
75%,240.750000,35.400000,25.200000,1.423044e+12,1.423044e+12,240.750000
max,324.000000,36.000000,34.200000,1.423128e+12,1.423128e+12,324.000000


In [23]:
sql_query = '''SELECT * FROM smartphone_sensor;'''
sensor_df = pd.read_sql_query(sql_query, conn)
sensor_df.describe()

,timestamp,AccelerationX,AccelerationY,AccelerationZ,MagneticFieldX,MagneticFieldY,MagneticFieldZ,Z-AxisAgle(Azimuth),X-AxisAngle(Pitch),Y-AxisAngle(Roll),GyroX,GyroY,GyroZ
count,1.835400e+04,18354.000000,18354.000000,18354.000000,18354.000000,18354.000000,18354.000000,18354.000000,18354.000000,18354.000000,18354.0,18354.0,18354.0
mean,1.422859e+12,-0.296746,4.211019,8.288839,-1.157590,-7.415054,-29.640253,168.779099,-25.079180,-1.687723,0.0,0.0,0.0
std,1.796786e+08,1.172366,2.326193,1.587541,16.907053,21.466757,11.524140,114.797401,20.257369,7.512564,0.0,0.0,0.0
min,1.422629e+12,-11.563000,-14.702000,-3.285000,-85.900000,-103.300000,-59.800000,0.005000,-114.158000,-79.969000,0.0,0.0,0.0
25%,1.422632e+12,-0.675000,3.196000,7.919000,-15.400000,-27.000000,-38.100000,46.692750,-35.234000,-3.859000,0.0,0.0,0.0
50%,1.422884e+12,-0.331000,4.057000,8.513000,-7.800000,-5.500000,-31.600000,143.816000,-25.197000,-1.936000,0.0,0.0,0.0
75%,1.423044e+12,0.014000,5.724000,8.991000,16.100000,9.300000,-23.300000,307.210250,-19.470000,0.071000,0.0,0.0,0.0
max,1.423128e+12,14.914000,12.781000,14.747000,37.900000,53.100000,37.200000,359.989000,158.389000,80.325000,0.0,0.0,0.0


In [24]:
sensor_df.head()

,timestamp,AccelerationX,AccelerationY,AccelerationZ,MagneticFieldX,MagneticFieldY,MagneticFieldZ,Z-AxisAgle(Azimuth),X-AxisAngle(Pitch),Y-AxisAngle(Roll),GyroX,GyroY,GyroZ
0,1422629483421,-3.577,2.104,9.078,-5.4,-17.9,-28.2,126.404,-12.169,-21.001,0,0,0
1,1422629483520,-3.424,2.927,9.374,-4.3,-21.9,-26.9,134.694,-11.322,-25.898,0,0,0
2,1422629483620,-2.505,2.908,8.867,-0.7,-24.4,-26.5,150.906,-17.523,-15.017,0,0,0
3,1422629483723,-5.473,1.472,9.394,0.3,-25.7,-26.9,138.022,-21.068,-23.941,0,0,0
4,1422629483822,-3.414,2.018,8.187,3.7,-29.6,-23.9,163.519,-12.814,-22.046,0,0,0


In [25]:
obj_feat = list(sensor_df.loc[:, sensor_df.dtypes == 'object'].columns.values)
obj_feat

[]

In [26]:
for feature in obj_feat:
    sensor_df[feature] = pd.to_numeric(sensor_df[feature])

In [27]:
sensor_df.head()

,timestamp,AccelerationX,AccelerationY,AccelerationZ,MagneticFieldX,MagneticFieldY,MagneticFieldZ,Z-AxisAgle(Azimuth),X-AxisAngle(Pitch),Y-AxisAngle(Roll),GyroX,GyroY,GyroZ
0,1422629483421,-3.577,2.104,9.078,-5.4,-17.9,-28.2,126.404,-12.169,-21.001,0,0,0
1,1422629483520,-3.424,2.927,9.374,-4.3,-21.9,-26.9,134.694,-11.322,-25.898,0,0,0
2,1422629483620,-2.505,2.908,8.867,-0.7,-24.4,-26.5,150.906,-17.523,-15.017,0,0,0
3,1422629483723,-5.473,1.472,9.394,0.3,-25.7,-26.9,138.022,-21.068,-23.941,0,0,0
4,1422629483822,-3.414,2.018,8.187,3.7,-29.6,-23.9,163.519,-12.814,-22.046,0,0,0


In [28]:
def train_validate_test_split(df, train_percent=.8, validate_percent=.2, seed=None, 
                              target="Z-AxisAgle(Azimuth)", categorical=["timestamp"]):
    
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    
    train_end = int(train_percent * m)
    train = df.iloc[perm[:train_end]].astype(float)
    validate = df.iloc[perm[train_end:]].astype(float)
    
    y_train = train[target]
    y_test = validate[target]
    
    train.drop([target]+categorical, axis=1, inplace=True)
    validate.drop([target]+categorical, axis=1, inplace=True)
    
    return train, y_train, validate, y_test

In [29]:
X_train, y_train, X_test, y_test = train_validate_test_split(sensor_df.copy())

In [30]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

dtrain = xgb.DMatrix(data=X_train, label=y_train)
dval = xgb.DMatrix(data=X_test, label=y_test)
# dtest = xgb.DMatrix(data=xtest)

params = dict(objective ='reg:linear', colsample_bytree = 0.2, learning_rate = 0.1,
            max_depth = 3, alpha = 10, n_estimators = 10, eval_metric='rmse', verbosity=2)

num_round = 50
bst = xgb.train(params, dtrain, num_round, evals=[(dtrain, 'dtrain'), (dval, 'dval')])

[05:40:34] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[05:40:34] WARNING: ../src/learner.cc:573: 
Parameters: { "n_estimators" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[05:40:34] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[0]	dtrain-rmse:189.34106	dval-rmse:191.06058
[05:40:34] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[1]	dtrain-rmse:176.71684	dval-rmse:178.37862
[05:40:34] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[2]	dtrain-rmse:166.35384	dval-rmse:167.94708
[05:40:35] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1

[05:40:35] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=3
[49]	dtrain-rmse:63.82523	dval-rmse:64.81294


In [31]:
# training
model = xgb.XGBRegressor(colsample_bytree = 0.2, learning_rate = 0.1,
            max_depth = 3, alpha = 10, n_estimators = 10, eval_metric='rmse', verbosity=2)

model.fit(X_train, y_train, 
          eval_set=[(X_train, y_train), (X_test, y_test)], 
          early_stopping_rounds=200)

[05:40:35] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[0]	validation_0-rmse:189.34106	validation_1-rmse:191.06058
[05:40:35] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[1]	validation_0-rmse:176.71684	validation_1-rmse:178.37862
[05:40:35] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[2]	validation_0-rmse:166.35384	validation_1-rmse:167.94707
[05:40:35] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[3]	validation_0-rmse:157.13200	validation_1-rmse:158.69766
[05:40:35] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[4]	validation_0-rmse:149.20567	validation_1-rmse:150.75369
[05:40:35] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[5]	validation_0-rmse:

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.2, eval_metric='rmse',
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=10, n_jobs=8,
             num_parallel_tree=1, random_state=0, reg_alpha=10, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=2)

In [32]:
# training
# save to JSON
model.save_model("model/model.json")

In [33]:
# load the model
model2 = xgb.XGBRegressor()
model2.load_model("model/model.json")

In [34]:
model2.predict(X_test)

array([101.70744 , 126.47478 , 111.85    , ...,  83.730446, 131.30318 ,
        87.99688 ], dtype=float32)

In [35]:
label_df.columns

Index(['ID', 'X', 'Y', 'arrival', 'departure', 'tid'], dtype='object')

In [36]:
# Trying querying a time stample and predict for that time stamp 

t1, t2 = label_df["arrival"][0], label_df["departure"][0]

In [47]:
sql_query = f'SELECT * FROM smartphone_sensor WHERE timestamp > {t1} AND timestamp <= {t2};'
sensor_df2 = pd.read_sql_query(sql_query, conn)
sensor_df2.describe()

,timestamp,AccelerationX,AccelerationY,AccelerationZ,MagneticFieldX,MagneticFieldY,MagneticFieldZ,Z-AxisAgle(Azimuth),X-AxisAngle(Pitch),Y-AxisAngle(Roll),GyroX,GyroY,GyroZ
count,4.200000e+01,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.00000,42.000000,42.000000,42.0,42.0,42.0
mean,1.423128e+12,-0.447952,5.894310,7.932048,-14.838095,-1.009524,-33.709524,40.55650,-36.345857,-2.425524,0.0,0.0,0.0
std,1.227355e+03,0.167519,0.135728,0.295671,0.342838,0.315308,0.257388,1.74828,1.149993,0.915897,0.0,0.0,0.0
min,1.423128e+12,-0.944000,5.388000,6.885000,-15.400000,-1.200000,-34.100000,37.30400,-39.150000,-4.660000,0.0,0.0,0.0
25%,1.423128e+12,-0.536250,5.821500,7.827750,-15.000000,-1.200000,-33.700000,39.51100,-36.892000,-2.990000,0.0,0.0,0.0
50%,1.423128e+12,-0.422000,5.886000,7.957500,-14.600000,-1.200000,-33.700000,40.24400,-36.398000,-2.310500,0.0,0.0,0.0
75%,1.423128e+12,-0.331000,5.979750,8.072000,-14.600000,-0.800000,-33.700000,41.55275,-35.635500,-1.861750,0.0,0.0,0.0
max,1.423128e+12,-0.206000,6.107000,8.752000,-14.600000,-0.200000,-33.300000,45.35000,-32.730000,-0.023000,0.0,0.0,0.0


In [38]:
sensor_df2.columns

Index(['timestamp', 'AccelerationX', 'AccelerationY', 'AccelerationZ',
       'MagneticFieldX', 'MagneticFieldY', 'MagneticFieldZ',
       'Z-AxisAgle(Azimuth)', 'X-AxisAngle(Pitch)', 'Y-AxisAngle(Roll)',
       'GyroX', 'GyroY', 'GyroZ'],
      dtype='object')

In [39]:
def handle_unique(df, target="Z-AxisAgle(Azimuth)", categorical=["timestamp"]):
    yval = df[target]
    df2 = df.drop([target]+categorical, axis=1)
    
    return df2, yval

xtest, yval = handle_unique(sensor_df2)
model2.predict(xtest)    

array([ 95.298164,  95.298164,  95.298164,  86.35084 ,  95.298164,
        95.298164,  95.298164,  88.33835 ,  88.33835 , 101.53775 ,
        97.28567 ,  95.298164,  88.33835 ,  88.33835 ,  97.28567 ,
        88.33835 ,  88.33835 ,  92.59043 ,  88.33835 ,  88.33835 ,
        95.298164,  87.6564  ,  86.35084 ,  87.6564  ,  86.35084 ,
        86.35084 ,  86.35084 ,  86.35084 ,  86.35084 ,  86.35084 ,
        86.35084 ,  86.35084 ,  86.35084 ,  86.35084 ,  86.35084 ,
        86.35084 ,  86.35084 ,  86.35084 ,  86.35084 ,  86.35084 ,
        86.35084 ,  95.298164], dtype=float32)

In [40]:
xtest.shape

(42, 11)

In [41]:
X_test.shape

(3671, 11)

In [42]:
len(label_df)

646

In [43]:
xgb.__version__

'1.4.2'